### This notebook shows how to convert from LGBM to ONNX

In [1]:
# Uncomment and run this cell if you need to install Hummingbird and ONNX
#!python -m pip install hummingbird_ml[extra,onnx]

In [2]:

import numpy as np
import lightgbm as lgb

from hummingbird.ml import convert


# Create some random data for binary classification.
num_classes = 2
X = np.array(np.random.rand(10000, 28), dtype=np.float32)
y = np.random.randint(num_classes, size=10000)

In [3]:
# Create and train a model (LightGBM in this case).
model = lgb.LGBMClassifier()
model.fit(X, y)

LGBMClassifier()

In [4]:
# Use Hummingbird to convert the ONNXML model to ONNX.
onnx_model = convert(model, "onnx", X)

Finished loading model, total used 100 iterations


In [5]:
%%timeit -r 3

# Run the ONNX model on CPU (if you have onnxruntime installed) or GPU (if you have onnxruntime-gpu installed)
onnx_model.predict(X)

502 ms ± 17.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
